<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EB%82%A8%EC%83%81%EC%9A%B0_%EC%9E%84%EC%8B%9C%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=0d9c5dcf087698673f1745b54e49ceb9a3f4ba63e8e5f0cfc307995e2156efee
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import torch
import torch.nn as nn
from torch import optim
from torch import optim

import warnings
%config InlineBackend.figure_format = 'retina'
warnings.filterwarnings("ignore")
%matplotlib inline

In [5]:
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/project_final/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

# 데이터 불러오기

In [6]:
pd.read_csv(f"{DATA_PATH}service1_9000_9000.csv").head(3)

,review,reply,sentiment
0,"맛이 별로였어요, 조리 상태도 별로였고 배달 시간도 오래 걸렸네요","안녕하세요~! 맛이 별로였다고 하셨는데, 제 기준에는 싱겁고 딱히 맛있어 보이지 않...",0
1,양도 많고 맛나요,하핫 정말 이런 이런 칭찬에 춤추고 싶네요 든든한 생각나실 때 또 만나요 친절한 고...,1
2,음식의 퀄리티가 낮아서 별로였어요. 맛이 없어서 실망했어요,음식의 퀄리티가 낮아서 실망하셨다니 정말 죄송합니다. 더 맛있는 음식으로 만족스러운...,0


In [7]:
train = pd.read_csv(f"{DATA_PATH}service1_9000_9000.csv")
train.isnull().sum()

review       0
reply        0
sentiment    0
dtype: int64

In [8]:
train['sentiment'].value_counts().sort_index() # 긍부정 분포 확인

0    9205
1    9205
Name: sentiment, dtype: int64

### 단문 데이터 제거
- 데이터 정제 과정에서 무의미해진 데이터는 학습데이터로 사용불가
    - 리뷰 : 공백만 생겼거나, 리뷰 이벤트를 위한 단답형 제거

In [15]:
train[(train['review'].str.len() > 4) & (train['review'].str.len() <= 10)] # 리뷰

,review,reply,sentiment
1,양도 많고 맛나요,하핫 정말 이런 이런 칭찬에 춤추고 싶네요 든든한 생각나실 때 또 만나요 친절한 고...,1
166,맛있어요 와플 추천,추천해 주셔서 감사합니다 맛있다는 말씀은 언제 들어도 설레는 것 같습니다 더욱더 연...,1
251,정말 맛있었습니다,주문을 통해 한걸음 가까워진 것 같아 기분이 좋아요 1 고객을 생각하는 마음 2 더...,1
256,맛있어서 2번 시킴,감사합니다 항상 좋은 재료로 음식을 만들겠습니다 안심하셔도 됩니다 더 좋은 모습으로...,1
357,양도 많고 맛있어요,좋은 재료를 사용하고 고객님께 좋은 평가 들으며 행복을 느낍니다 고객님들 맛있게 드...,1
...,...,...,...
18150,맛있게 먹었습니다,저희 항정살 맛있게 잘 드시고 소중한 리뷰도 남겨 주셔서 너무 기쁘고 보람됩니다 저...,1
18164,맛있어요 괜찮아요,건강한 한 끼 저렴하지만 맛있게 만들겠습니다 저희 찾아주셔서 감사합니다,1
18178,맛이 너무 없어요.,불편을 드려 대단히 죄송합니다. 귀중한 의견 감사드립니다.,0
18197,맛있어요 2번째임,감사합니다 부탁드려요 진심으로 밈 맘에 들었다면 찜 부탁드립니다,1


In [21]:
train[(train['review'].str.len() > 4) & (train['review'].str.len() <= 10)] # 리뷰

,review,reply,sentiment
1,양도 많고 맛나요,하핫 정말 이런 이런 칭찬에 춤추고 싶네요 든든한 생각나실 때 또 만나요 친절한 고...,1
166,맛있어요 와플 추천,추천해 주셔서 감사합니다 맛있다는 말씀은 언제 들어도 설레는 것 같습니다 더욱더 연...,1
251,정말 맛있었습니다,주문을 통해 한걸음 가까워진 것 같아 기분이 좋아요 1 고객을 생각하는 마음 2 더...,1
256,맛있어서 2번 시킴,감사합니다 항상 좋은 재료로 음식을 만들겠습니다 안심하셔도 됩니다 더 좋은 모습으로...,1
357,양도 많고 맛있어요,좋은 재료를 사용하고 고객님께 좋은 평가 들으며 행복을 느낍니다 고객님들 맛있게 드...,1
...,...,...,...
18150,맛있게 먹었습니다,저희 항정살 맛있게 잘 드시고 소중한 리뷰도 남겨 주셔서 너무 기쁘고 보람됩니다 저...,1
18164,맛있어요 괜찮아요,건강한 한 끼 저렴하지만 맛있게 만들겠습니다 저희 찾아주셔서 감사합니다,1
18178,맛이 너무 없어요.,불편을 드려 대단히 죄송합니다. 귀중한 의견 감사드립니다.,0
18197,맛있어요 2번째임,감사합니다 부탁드려요 진심으로 밈 맘에 들었다면 찜 부탁드립니다,1


In [23]:
len(train[train['review'].str.len() <= 10])

269

In [24]:
train = train[train['review'].str.len() > 10]
train

,review,reply,sentiment
0,"맛이 별로였어요, 조리 상태도 별로였고 배달 시간도 오래 걸렸네요","안녕하세요~! 맛이 별로였다고 하셨는데, 제 기준에는 싱겁고 딱히 맛있어 보이지 않...",0
2,음식의 퀄리티가 낮아서 별로였어요. 맛이 없어서 실망했어요,음식의 퀄리티가 낮아서 실망하셨다니 정말 죄송합니다. 더 맛있는 음식으로 만족스러운...,0
3,리뷰가 좋아서 주문했는데 정말 맛있게 잘 먹었습니다 다 맛있어요 김밥 완전 빅 사이...,감사합니다 언제나 푸짐한 양과 맛있는 맛으로 찾아뵙겠습니다 자주 애용해 주세요,1
4,항상 맛있게 잘 먹고 있습니다,고객님 소중한 고객님이 이임 항상 찾아주시고 맛있게 드셔주시니 너어어어무 기쁘고 행...,1
5,음식이 맛이 없어서 좀 아쉬웠어요. 다음에는 다른 음식점에서 시켜야겠어요,저희 음식이 맛이 없어서 아쉬움을 드렸군요. 다음에는 고객님께 맛있는 음식을 제공할...,0
...,...,...,...
18405,친구시켜줬는데 사진은 이렇지만 고기 많고 맛있대요 감사합니다,저희 매장을 빛내 주셔서 영광 맛깔나는 사진 너무 감사해요 맛있게 드셔 주시기만 해...,1
18406,맛있게 먹는 단골집,자주자주 찾아주세요 저희 이용해 주셔서 감사합니다 최고보다는 최선을 다하는 될 수 ...,1
18407,가격이 좀 비싸서 아쉬웠어요. 좀 더 저렴하면 좋겠어요,가격이 비싸셨다니 정말 죄송합니다. 고객님의 의견을 반영하여 가격을 조정할 수 있도...,0
18408,센스가 없는 배달이었어요.,다음에는 더욱더 만족스러운 서비스를 제공하기 위해 노력하겠습니다.,0


## 긍부정 데이터 비율 선택

In [25]:
train['sentiment'].value_counts().sort_index() # 긍부정 분포 확인

0    9183
1    8958
Name: sentiment, dtype: int64

In [26]:
train_neg = train[train['sentiment'] == 0]

# 긍정 데이터 중 중복된 답변제거
train_pos = train[train['sentiment'] == 1].drop_duplicates(subset='reply', keep=False)
train_pos['sentiment'].value_counts().sort_index()

1    8958
Name: sentiment, dtype: int64

In [ ]:
length = len(train_neg)
length

9205

In [28]:
# 긍정 데이터를 랜덤으로 추출하기 위해 섞기
shuffled_train_pos = train_pos.sample(frac=1, random_state=42)
shuffled_train_neg = train_neg.sample(frac=1, random_state=42)

train_pos = shuffled_train_pos.iloc[:50].reset_index(drop=True)
train_neg = shuffled_train_neg.iloc[:50].reset_index(drop=True)

# 데이터 합치기
train = pd.concat([train_neg, train_pos], ignore_index=True)
train['sentiment'].value_counts().sort_index()

0    50
1    50
Name: sentiment, dtype: int64

In [29]:
# 다시 섞기
train = train.sample(frac=1, random_state=42).reset_index(drop=True)
train

,review,reply,sentiment
0,역시 회는 윤희 횟집,항상 저희 찾아주셔서 감사드립니다 앞으로도 저희 고객님께서 맛있게 드실 수 있게 최...,1
1,역시나 믿고 시키는 마라샹궈 맛집입니다 또 시켜 먹을 예정 모두 믿고 한 번만 시켜...,안녕하세요 저희 매장 찾아주시어 감사드립니다 아무리 힘든 날에도 맛난 메뉴가 입안 ...,1
2,맛있게 잘 먹었습니다 양도 많고 좋아요,고객님 맛있게 드셔주셔서 감사합니다 앞으로도 맛있는 음식으로 보답하겠습니다 감사합니다,1
3,맛은 그럭저럭 이지만 배달 비용이 비싸서 좀 아쉬웠습니다. 다른 곳을 찾아보려고 생...,맛은 그럭저럭인데 배달 비용을 아쉬워하신다니 저희 매장으로서도 회원님의 의견을 소중...,0
4,전체적으로 아쉬웠어요. 음식 맛도 별로고 서비스도 별로였어요.,아쉬움을 드려 죄송합니다. 맛과 서비스 모두 향상시킬 수 있도록 노력하겠습니다.,0
...,...,...,...
95,매번 잘 먹고 있습니다,진심으로 감사합니다 항상 이벤트 아닐 때도 리뷰 남겨주셔서 아이디 2글자를 잘 기억...,1
96,늘 잘 먹고 있어요 다 맛있어요,감사합니다 고객님 맛있게 해드릴게요,1
97,배달 시간이 오래 걸렸고 음식이 따뜻하지 않았어요.,배달 시간과 음식의 상태에 불편을 드려 정말 죄송합니다. 다음에는 빠르고 따뜻한 음...,0
98,배달도 빠르고 맛있어요,저희를 이렇게나 좋아해 주시다니 너무 감사합니다 항상 최선을 다하는 되겠습니다 좋은...,1


# 유사도 확인

## 유사도 군집화 라벨 불러오기

In [36]:
pd.read_csv(f"{DATA_PATH}sim_cluster_label.csv")

,label
0,맛이 없어요
1,양이 적어요
2,배달이 느려요
3,맛은 있는데 양이 적어요
4,맛은 있는데 배달이 느려요
5,양은 많은데 맛이 없어요
6,배달은 빠른데 맛이 없어요
7,양은 많은데 배달이 느려요
8,배달은 빠른데 양이 적어요
9,맛있어요


In [37]:
sim_cluster_label = pd.read_csv(f"{DATA_PATH}sim_cluster_label.csv")

In [32]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import jaccard_score
bert_model = SentenceTransformer('jhgan/ko-sroberta-multitask')

def evaluate_similarity(label_text, review_text, alpha=0.9): # cosine 가중치 조절
    label_embedding = bert_model.encode(label_text)
    review_embedding = bert_model.encode(review_text)

    # 코사인 유사도
    cosine_sim = 1 - pairwise_distances([label_embedding], [review_embedding], metric='cosine')[0][0]

    # 자카드 유사도
    label_tokens = set(label_text.split())
    review_tokens = set(review_text.split())
    jaccard_sim = len(label_tokens.intersection(review_tokens)) / len(label_tokens.union(review_tokens))

    # 가중 평균 내보기
    weighted_sim = alpha*cosine_sim + (1 - alpha)*jaccard_sim

    return weighted_sim

In [53]:
result = []
for label in sim_cluster_label['label']:
    for review in train['review']:
        score = evaluate_similarity(label, review)
        if score >= 0.5:
            result.append({'label': label, 'review': review, 'score': score})

result = pd.DataFrame(result)
result

,label,review,score
0,맛이 없어요,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...",0.579780
1,맛이 없어요,맛이 별로였어요. 가격대비 너무 아쉬웠어요,0.538105
2,맛이 없어요,맛이 너무 싱겁고 느끼해서 별로였어요. 좀 더 강한 맛이 있었으면 좋았을 텐데.,0.538126
3,맛이 없어요,별로 맛있지 않았고 배송이 늦어서 식어서 먹기도 그렇게 좋지 않았어요.,0.535679
4,맛이 없어요,맛이 너무 떨어지고 식감도 별로였어요. 다음에는 다른 가게에서 시킬게요,0.571493
...,...,...,...
132,양이 많아요,양도 많고 맛있고 반찬도 좋아요,0.640549
133,양이 많아요,양도 푸짐하게 주시고 맛있게 잘 먹었습니다,0.507493
134,배달이 빨라요,맛도 좋고 배달도 빠르고 좋아요,0.714499
135,배달이 빨라요,맛있어요 배달도 빠르고요 날 더울 땐 7번가 피자,0.635382


In [54]:
# 엑셀로 확인
with pd.ExcelWriter("result.xlsx", engine='openpyxl') as writer:
    result.to_excel(writer, sheet_name='시트1', index=False)